In [1]:
### Imports ###

import nltk
import re
import string
import os
import pandas as pd
import numpy as np
import ast
import spacy
import re
import string
from spacy.lang.en import English
import nltk

### Changing the work directory ###

"""
Kindly change your working directory here 
"""
os.chdir("D:\\UniSaarland\\incar-dialogue")

In [2]:
### Uploading Datasets ###

train_df = pd.read_csv("train.csv")
valid_df = pd.read_csv("valid.csv")
test_df = pd.read_csv("test.csv")

DFs = [train_df,valid_df,test_df]

In [3]:
### Utility functions for text cleaning ###

"""
These functions will help to clean the corpus.
---NOT USED ---
I am not using these functions here as we will have to clean the KG as well in order to use the clean utterances as
the entities in the kg have special characters and punctuations with them
"""

# def clean_text(x):
#     text = re.sub('(\d+)','',x)   
#     text = text.lower()
#     return text
# def remove_url(x):
#     text = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})\/([a-zA-Z0-9_]+]*)',' ',x)
#     return text
# def remove_punct(x):
#     text_without_puct = [t for t in x if t not in string.punctuation]
#     text_without_puct = ''.join(text_without_puct)
#     return text_without_puct

### Cleaning the dataframes ###

# for df in DFs:
    
#     df["utterance"] = df["utterance"].apply(clean_text)
#     df["utterance"] = df["utterance"].apply(remove_url)
#     df["utterance"] = df["utterance"].apply(remove_punct)
#     df["context"] = df["context"].apply(clean_text)
#     df["context"] = df["context"].apply(remove_url)
#     df["context"] = df["context"].apply(remove_punct)


'\nThese functions will help to clean the corpus.\n---NOT USED ---\nI am not using these functions here as we will have to clean the KG as well in order to use the clean utterances as\nthe entities in the kg have special characters and punctuations with them\n'

In [4]:
def match(kg_triples,entList):
    """
    Utility function for PredictEntities
    Input : knowledge-graph's triples and extracted triples from the paragraph
    Returns : Predicted triples 
    """
    extracted_triples =[]
    for ent in entList:
        for triple in kg_triples:
            if ent == triple[0]:
                extracted_triples.append(triple)
    return extracted_triples

In [5]:
def PredictTriples(dataframe):
    """
    This is the main function that performs the triple predictions after extracting triples from the utterances 
    Input : The dataframe from which triples are to be extracted an is to be evaluated
    Returns : Predicted triples from each row of the dataframe 
    
    Note : Please not that as recommended in the problem, I have only used utterance, context and kg as inputs 
    """
    
    NER = spacy.load("en_core_web_sm")
    df = dataframe
    predicted_labels=[]
    for index, row in df.iterrows():
        paragraph = []
        row["context"] = ast.literal_eval(row["context"])
        for utterance in row["context"]:
            paragraph.append(utterance)
        paragraph.append(row["utterance"])
        text = ' '.join(map(str, paragraph))
        text1 = NER(text)
        entList = []
        for word in text1.ents:
            entList.append(word.text)
        kg_triples = ast.literal_eval(row["kg"])
        predicted_labels.append(match(kg_triples,entList)) # function call to match()
        if index % 100 == 0 and index != 0: 
            print("{} values extracted!".format(index))
    return predicted_labels   

In [6]:
def CalculateAccuracy(predicted_labels,ground_labels):
    
    """
    This function calculates instance/row wise score(accuracy)
    Input : predicted-labels and ground-lables
    Returns : A list with accuracy for each row 
    """
    instance_accuracy = []
    for idx in range(len(predicted_labels)): 
        ground_val = ast.literal_eval(ground_labels.iloc[idx])
        intersection_vals = [set(tuple(row) for row in predicted_labels[idx] ) & set(tuple(row) for row in ground_val)]
        if intersection_vals == [set()]:
            instance_accuracy.append(0)
        else:
            if len(ground_val) == 0:
                instance_accuracy.append(0)
            else:
                score = (len(intersection_vals))/len(ground_val)
                instance_accuracy.append(score*100)

    return instance_accuracy

In [7]:
### Driver Section ###

"""
Please select the dataframe you want to evaluate from - train_df, valid_df,test_df

"""

print("##### Processing and evaluating train_df #####")
df = train_df
df_name = "train_df"
predicted_labels = PredictTriples(df)
ground_labels = df.labels
if len(predicted_labels) == len(ground_labels):
    instance_accuracy = CalculateAccuracy(predicted_labels,ground_labels)
else :
    print("Dimension mismatch between the predicted and ground labels")
average_accuracy = round((sum(instance_accuracy)/len(instance_accuracy)),2)

#print(predicted_labels)
#print(instance_accuracy)
print("The average accuracy for the dataset {} is {} %.".format(df_name,average_accuracy))

################################################################################################################

print("##### Processing and evaluating valid_df #####")
df = valid_df
df_name = "valid_df"
predicted_labels = PredictTriples(df)
ground_labels = df.labels
if len(predicted_labels) == len(ground_labels):
    instance_accuracy = CalculateAccuracy(predicted_labels,ground_labels)
else :
    print("Dimension mismatch between the predicted and ground labels")
average_accuracy = round((sum(instance_accuracy)/len(instance_accuracy)),2)

#print(predicted_labels)
#print(instance_accuracy)
print("The average accuracy for the dataset {} is {} %.".format(df_name,average_accuracy))


###############################################################################################################


print("##### Processing and evaluating test_df #####")
df = test_df
df_name = "test_df"
predicted_labels = PredictTriples(df)
ground_labels = df.labels
if len(predicted_labels) == len(ground_labels):
    instance_accuracy = CalculateAccuracy(predicted_labels,ground_labels)
else :
    print("Dimension mismatch between the predicted and ground labels")
average_accuracy = round((sum(instance_accuracy)/len(instance_accuracy)),2)

#print(predicted_labels)
#print(instance_accuracy)
print("The average accuracy for the dataset {} is {} %.".format(df_name,average_accuracy))

##### Processing and evaluating train_df #####
100 values extracted!
200 values extracted!
300 values extracted!
400 values extracted!
500 values extracted!
600 values extracted!
700 values extracted!
800 values extracted!
900 values extracted!
1000 values extracted!
1100 values extracted!
1200 values extracted!
1300 values extracted!
1400 values extracted!
1500 values extracted!
1600 values extracted!
1700 values extracted!
1800 values extracted!
1900 values extracted!
2000 values extracted!
2100 values extracted!
2200 values extracted!
2300 values extracted!
2400 values extracted!
2500 values extracted!
2600 values extracted!
2700 values extracted!
2800 values extracted!
2900 values extracted!
3000 values extracted!
3100 values extracted!
3200 values extracted!
3300 values extracted!
3400 values extracted!
3500 values extracted!
3600 values extracted!
3700 values extracted!
3800 values extracted!
3900 values extracted!
4000 values extracted!
4100 values extracted!
4200 values extract